# Train Football Object Detection Model:

---

Combined Dataset:
- Player and Football:

    <a href="https://universe.roboflow.com/private-uofaa/football-wli4r">
        <img src="https://app.roboflow.com/images/download-dataset-badge.svg"></img>
    </a>



Seperate Datasets:
- Footballs:
  
    <a href="https://universe.roboflow.com/private-uofaa/football-balls-detection">
        <img src="https://app.roboflow.com/images/download-dataset-badge.svg"></img>
    </a>
- Players:
  
    <a href="https://universe.roboflow.com/private-uofaa/footballplayers-4f1na">
        <img src="https://app.roboflow.com/images/download-dataset-badge.svg"></img>
    </a>

This notebook is based on the [How to Train YOLOv8 Object Detection on a Custom Dataset](https://github.com/roboflow/notebooks/blob/main/notebooks/train-yolov8-object-detection-on-custom-dataset.ipynb) notebook from the [roboflow/notebooks](https://github.com/roboflow/notebooks) repository.

## Configure API keys

- Open your [`Roboflow Settings`](https://app.roboflow.com/settings/api) page. Click `Copy`. This will place your private key in the clipboard.
- In Colab, go to the left pane and click on `Secrets` (🔑). Store Roboflow API Key under the name `ROBOFLOW_API_KEY`.

## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Settings` -> `Accelorator` and set it to `GPU T4x2`.

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Install all required dependencies:

In [ ]:
!pip3 install -q ultralytics roboflow

## Imports required packages and libraries:

In [ ]:
from roboflow import Roboflow
from google.colab import userdata
from IPython.display import Image

## Pull the dataset upon which it will be trained and testeted:

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
ROBOFLOW_API_KEY = user_secrets.get_secret("Roboflow")

# Use the following line if working with Google Collab:
# ROBOFLOW_API_KEY = userdata.get('Roboflow')

# Use the following line if working with Kaggle:  
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

# Dataset for combined model:
project = rf.workspace("private-uofaa").project("football-wli4r")
version = project.version(1)
dataset = version.download("yolov8")

# Datasets for seperate models:
# project = rf.workspace("private-uofaa").project("footballplayers-4f1na")
# version = project.version(2)
# players_dataset = version.download("yolov8")
# project = rf.workspace("private-uofaa").project("football-balls-detection")
# version = project.version(1)
# ball_dataset = version.download("yolov8")

**NOTE:** Update `data.yaml` structure to align with the latest requirements of the `ultralytics` package.



In [ ]:
!sed -i 's|\(train: \).*|\1../train/images|' {dataset.location}/data.yaml
!sed -i 's|\(val: \).*|\1../valid/images|' {dataset.location}/data.yaml

In [ ]:
# !sed -i 's|\(train: \).*|\1../train/images|' {players_dataset.location}/data.yaml
# !sed -i 's|\(val: \).*|\1../valid/images|' {players_dataset.location}/data.yaml
# !sed -i 's|\(train: \).*|\1../train/images|' {ball_dataset.location}/data.yaml
# !sed -i 's|\(val: \).*|\1../valid/images|' {ball_dataset.location}/data.yaml

## Custom training:

In [ ]:
# Code to train one combined model that detects footballs, goalkeepers, players and referees.
# Adjust the batch number according to the amount of GPU you have available.
%cd {HOME}

!yolo task=detect mode=train model=yolov8x.pt data={dataset.location}/data.yaml batch=8 epochs=50 imgsz=1280 plots=True device=0,1 patience=5 verbose=True

In [ ]:
# Code to train two seperate models. One for footballs and one for goalkeepers, players and referees.
# Adjust the batch number according to the amount of GPU you have available.
# %cd {HOME}

# !yolo task=detect mode=train model=yolov8x.pt data={players_dataset.location}/data.yaml batch=8 epochs=50 imgsz=1280 plots=True device=0,1 patience=5 verbose=True

In [ ]:
# Code to train two seperate models. One for footballs and one for goalkeepers, players and referees.
# Adjust the batch number according to the amount of GPU you have available.
# %cd {HOME}

# !yolo task=detect mode=train model=yolov8x.pt data={ball_dataset.location}/data.yaml batch=8 epochs=50 imgsz=1280 plots=True device=0,1 patience=5 verbose=True

In [ ]:
!ls {HOME}/runs/detect/train/

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

## Validate custom model

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train6/weights/best.pt data={dataset.location}/data.yaml imgsz=1280

## Deploy model on Roboflow:

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
ROBOFLOW_API_KEY = user_secrets.get_secret("Roboflow")


# ROBOFLOW_API_KEY = userdata.get('Roboflow')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)


project = rf.workspace("private-uofaa").project("football-detection-ysgxf")
version = project.version(3)
dataset = version.download("yolov8")

In [ ]:
project.version(dataset.version).deploy(model_type="yolov8", model_path=f"{HOME}/runs/detect/train/")